In [ ]:
## https://www.chc.ucsb.edu/data/chirps3

In [2]:
import os
import requests
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from osgeo import gdal
import glob
import shutil

In [9]:
years = [ano for ano in range(1985, 2024) if ano not in [1985, 1993, 2003, 2013, 2023]]
months = list(range(1, 13))

In [4]:
target_resolution = 30
target_crs_utm = "EPSG:31983"

In [5]:
os.makedirs(f"./chirps_v3_latam/Precipitation_monthly/", exist_ok=True)

In [6]:
def resample_raster(folder, raster_file, target_crs_utm, target_resolution, nodata):  
    raster_path = os.path.join(folder, raster_file)
    if not os.path.exists(raster_path):
        print(f"Arquivo não encontrado: {raster_path}")
        return
    outfn = os.path.join(folder, "resampled_" + raster_file)
    ds = gdal.Open(raster_path)
    if ds is None:
        print(f"Falha ao abrir o arquivo {raster_file}")
        return
   
    proj_ds = gdal.Warp('', ds, dstSRS=target_crs_utm, format='MEM')
    if proj_ds is None:
        print(f"Falha na reprojeção do arquivo {raster_file}")
        return
    resample_alg = 'near'
    ds_resampled = gdal.Warp(outfn, proj_ds, xRes=target_resolution, yRes=target_resolution, resampleAlg=resample_alg, dstNodata=nodata)
    if ds_resampled is None:
        print(f"Falha na operação de resampling para o arquivo {raster_file}")
        return
    print(f"Processado: {raster_path} para {outfn}")
    ds_resampled = None
    proj_ds = None
    ds = None



def crop_raster(folder, raster_file, roi):
    raster_path = os.path.join(folder, raster_file)
    with rasterio.open(raster_path) as src:
        roi = roi.to_crs(src.crs)
        shapes = [geom.__geo_interface__ for geom in roi.geometry]
        out_image, out_transform = mask(src, shapes, crop=True, all_touched = True)
        out_meta = src.meta.copy()
        out_meta.update({
        "driver": "GTiff",
        "count": 1,  
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform
    })
    output_raster_path_utm = os.path.join(folder, f"cropped_{raster_file}")
    with rasterio.open(output_raster_path_utm, 'w', **out_meta) as dest:
        dest.write(out_image[0], 1)

      
    print(f"Processado e salvo: {output_raster_path_utm}")

In [7]:
bh = gpd.read_file("../bh_lago_paranoa.geojson")
bh_30 = bh.copy()
bh_6000 = bh.copy()

bh_6000 = bh_6000.to_crs("EPSG:31983")
bh_6000.geometry = bh_6000.geometry.buffer(6000)
bh_6000 = bh_6000.to_crs("EPSG:4326")

bh_30 = bh_30.to_crs("EPSG:31983")
bh_30.geometry = bh_30.geometry.buffer(30)
bh_30 = bh_30.to_crs("EPSG:4326")

In [10]:
chirps_orig_files = []

for year in years:
    for month in months:

        ## DOWNLOAD DOS ARQUIVOS DO SITE DO CHIRPS V3
        month_str = f"{month:02d}"
        
        url = f"https://data.chc.ucsb.edu/products/CHIRPS/v3.0/monthly/latam/tifs/chirps-v3.0.{year}.{month_str}.tif"
        file_name = f"chirps-v3.0.{year}.{month_str}.tif"
        
        output_dir = f"chirps_v3_latam/{year}"
        os.makedirs(output_dir, exist_ok=True)
        file_path = os.path.join(output_dir, file_name)
        
        response = requests.get(url)
        if response.status_code == 200:
            with open(file_path, 'wb') as f:
                f.write(response.content)
                chirps_orig_files.append(file_path)
            print(f"Downloaded: {file_name}")
        else:
            print(f"Failed to download: {file_name}")
        
        ## RECORTA O DADO LATAM PARA O ROI + BUFFER 30M
        crop_raster(output_dir, file_name, bh_6000)
        ## REAMOSTRA PARA A RES. ESP. 30 M
        resample_raster(output_dir, f"cropped_{file_name}", target_crs_utm, target_resolution, 0)
        ## RECORTA PRO ROI CERTINHO
        crop_raster(output_dir, f"resampled_cropped_{file_name}", bh)
        
        src_path = os.path.join(output_dir, f"cropped_resampled_cropped_{file_name}")
        dst_path = os.path.join(f"./chirps_v3_latam/Precipitation_monthly/{year}", file_name)
        os.makedirs(f"./chirps_v3_latam/Precipitation_monthly/{year}/", exist_ok=True)
        shutil.copy(src_path, dst_path)

        print(f"Arquivo copiado para: {dst_path}")
       


Downloaded: chirps-v3.0.2024.01.tif
Processado e salvo: chirps_v3_latam/2024/cropped_chirps-v3.0.2024.01.tif


/usr/lib/python3/dist-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Processado: chirps_v3_latam/2024/cropped_chirps-v3.0.2024.01.tif para chirps_v3_latam/2024/resampled_cropped_chirps-v3.0.2024.01.tif
Processado e salvo: chirps_v3_latam/2024/cropped_resampled_cropped_chirps-v3.0.2024.01.tif
Arquivo copiado para: ./chirps_v3_latam/Precipitation_monthly/2024/chirps-v3.0.2024.01.tif
Downloaded: chirps-v3.0.2024.02.tif
Processado e salvo: chirps_v3_latam/2024/cropped_chirps-v3.0.2024.02.tif
Processado: chirps_v3_latam/2024/cropped_chirps-v3.0.2024.02.tif para chirps_v3_latam/2024/resampled_cropped_chirps-v3.0.2024.02.tif
Processado e salvo: chirps_v3_latam/2024/cropped_resampled_cropped_chirps-v3.0.2024.02.tif
Arquivo copiado para: ./chirps_v3_latam/Precipitation_monthly/2024/chirps-v3.0.2024.02.tif
Downloaded: chirps-v3.0.2024.03.tif
Processado e salvo: chirps_v3_latam/2024/cropped_chirps-v3.0.2024.03.tif
Processado: chirps_v3_latam/2024/cropped_chirps-v3.0.2024.03.tif para chirps_v3_latam/2024/resampled_cropped_chirps-v3.0.2024.03.tif
Processado e salvo:

In [12]:
def check_data(raster_file):
    with rasterio.open(raster_file) as src:  
        largura = src.width
        altura = src.height
        resolucao = src.res
        metadata = src.meta
        print(f"Largura (Width): {largura}")
        print(f"Altura (Height): {altura}")
        print(f"Resolução espacial (Spatial resolution): {resolucao}")
        print("Metadata (src):")
        for key, value in metadata.items():
            print(f"{key}: {value}")

check_data("./chirps_v3_latam/Precipitation_monthly/2024/chirps-v3.0.2024.12.tif")

Largura (Width): 1449
Altura (Height): 1514
Resolução espacial (Spatial resolution): (30.0, 30.0)
Metadata (src):
driver: GTiff
dtype: float32
nodata: 0.0
width: 1449
height: 1514
count: 1
crs: EPSG:31983
transform: | 30.00, 0.00, 167074.58|
| 0.00,-30.00, 8275948.29|
| 0.00, 0.00, 1.00|


In [13]:
check_data("./chirps_v3_latam/Precipitation_monthly/2023/chirps-v3.0.2023.12.tif")

Largura (Width): 1449
Altura (Height): 1514
Resolução espacial (Spatial resolution): (30.0, 30.0)
Metadata (src):
driver: GTiff
dtype: float32
nodata: 0.0
width: 1449
height: 1514
count: 1
crs: EPSG:31983
transform: | 30.00, 0.00, 167074.58|
| 0.00,-30.00, 8275948.29|
| 0.00, 0.00, 1.00|
